## The Principle: Trigonometry
The idea here is simple, given (accurate, a big ask) metric depth values for every pixel's ray, allows us to calculate the length of any line segment in the 3D world which has its endpoints visible, using trigonometry. The depth values for the corresponding pixels give us the length of two sides of the triangle, and UniDepth's dense camera prediction directly gives us the angle between the two lines (without having to figure out the FOV).

Armed with the length of two sides and the measure of their contained angle, I'm 99.999% sure we can compute the third side, although I've never been very good at trigonometry and keep forgetting the law of cosines.

Unidepth makes this even easier for us, since it predicts rays completely using its pseudo-spherical output space, it can directly give us the world points corresponding to each pixel (calculated using its predicted camera parameters), which we can just calculate the euclidean distance between.

In [1]:
from unidepth.models import UniDepthV2

c:\ProgramData\miniconda3\envs\cv-proj\lib\site-packages\timm\models\layers\__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
A matching Triton is not available, some optimizations will not be enabled
Traceback (most recent call last):
  File "c:\ProgramData\miniconda3\envs\cv-proj\lib\site-packages\xformers\__init__.py", line 55, in _is_triton_available
    from xformers.triton.softmax import softmax as triton_softmax  # noqa
  File "c:\ProgramData\miniconda3\envs\cv-proj\lib\site-packages\xformers\triton\softmax.py", line 11, in <module>
    import triton
ModuleNotFoundError: No module named 'triton'
Triton is not available, some optimizations will not be enabled.
This is just a warning: triton is not available


In [2]:
model = UniDepthV2.from_pretrained("lpiccinelli/unidepth-v2-vitl14")

In [3]:
import torch

dev = "cuda:0" if torch.cuda.is_available() else "cpu"
model = model.to(dev)

First, I generate depth maps + camera preds for each guy

In [4]:
import os
import PIL.Image as Image
import matplotlib.pyplot as plt
from tqdm import tqdm

import numpy as np

IMG_DIR = "data"
OUT_DIR = "unidepth_out"

depths = {}
points = {}

for filename in tqdm([s for s in os.listdir(IMG_DIR) if s.endswith(".jpg")]):
	name = os.path.splitext(filename)[0]
	
	img = np.array(Image.open(os.path.join(IMG_DIR, filename)))
	img_torch = torch.from_numpy(img).permute((2, 0, 1))

	preds = model.infer(img_torch)

	depth = np.fliplr(preds["depth"][0].squeeze().cpu().numpy().transpose())

	depths[name] = depth
	points[name] = preds["points"].squeeze().cpu()

	plt.imsave(os.path.join(OUT_DIR, f"{name}.png"), depth, cmap="gray")

  0%|          | 0/24 [00:00<?, ?it/s]D:\ASPDrive\AcademicWork\Sem7\CV\workspace\project\cv-project\UniDepth\unidepth\models\unidepthv2\unidepthv2.py:47: UserWarning: Resolution level is not set. Using max resolution. You can tradeoff resolution for speed by setting a number in [0,10]. This can be achieved by setting model's `resolution_level` attribute.
  warnings.warn(
D:\ASPDrive\AcademicWork\Sem7\CV\workspace\project\cv-project\UniDepth\unidepth\layers\attention.py:142: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  x = F.scaled_dot_product_attention(
100%|██████████| 24/24 [00:50<00:00,  2.12s/it]


In [5]:
np.savez(f"{OUT_DIR}/depths.npz", **depths)
np.savez(f"{OUT_DIR}/points.npz", **points)

In [6]:
points = np.load(f"{OUT_DIR}/points.npz")

points["kartripta1"].shape

(3, 3072, 4096)

Before we begin, I want to talk about one neat benefit that Criminisi's method has over this one. Criminisi does not actually use any image data. It is a purely geometric derivation, and therefore, is not fazed by visual characteristics such as transparency, lighting conditions, etc etc. It is only concerned with projective invariants, and the only "visual" aspect of it is for identification of the keypoints.

On the other hand, depth estimation, predictably, is very sensitive to these image characteristics, since it has nothing else to go off of. This results in outputs like these:

<center>
	<img src = "data/kartripta9.jpg" style="width: 30%">
	<img src = "unidepth_out/kartripta9.png" style="width: 30%">
</center>

Clearly, it seems to register some of the glass wall as an actual solid wall, which means that this method won't work on this image. For Criminisi, however, this image is an ideal case, with the image plane at a high inclination angle to the world axis, we see it performing extremely well.